In [1]:
def getNode(name, l):
   return next(( i for i in l if i.name == name), -1)

In [2]:
from prettytable import PrettyTable 

class Node:
   def __init__(self, name):
       self.parent = 0
       self.name = name
       self.edges = []
       self.value = 0
           
   def __lt__(self, other):
      return self.value < other.value
     

class Edge:
   def __init__(self, edge): #INPUT E.G. edge = ('Arad', 'Sibiu', 140)
      self.start = edge[0]
      self.end = edge[1]
      self.value = edge[2]
      
   def __lt__(self, other):
      return self.value < other.value


class Graph:
   def __init__(self, node_list, edges):
      self.nodes = []
      for name in node_list:
         self.nodes.append(Node(name)) #MAKES A LIST OF ALL NODES

      for e in edges:
        e = (getNode(e[0],self.nodes), getNode(e[1], self.nodes), e[2]) # PUTS REAL NODES INTO THE EDGES 

        self.nodes[next((i for i,v in enumerate(self.nodes) if v.name == e[0].name), -1)].edges.append(Edge(e)) #ADDS EDGES TO THE NODES 
        self.nodes[next((i for i,v in enumerate(self.nodes) if v.name == e[1].name), -1)].edges.append(Edge((e[1], e[0], e[2]))) #ADDS EDGES TO THE NODES 


   def print(self):
      node_list = self.nodes
      
      t = PrettyTable(['  '] +[i.name for i in node_list])
      for node in node_list:
         edge_values = ['X'] * len(node_list)
         for edge in node.edges:
            edge_values[ next((i for i,e in enumerate(node_list) if e.name == edge.end.name) , -1)] = edge.value           
         t.add_row([node.name] + edge_values)
      print(t)
      
      
   def contains(self, node):
      if isinstance(node, str):
         return any(n.name == node for n in self.nodes)
      if isinstance(node, Node):
         return any(n.name == node.name for n in self.nodes)

   
   def get(self, node_name):
      return next((node for node in self.nodes if node.name == node_name), None)
   
   def print_path(self, goal_node):
      path = []
      total_cost = 0
      current = goal_node

      while current:
         path.append(current.name)
         if current.parent:
            for edge in current.parent.edges:
               if edge.end.name == current.name:
                  total_cost += edge.value
                  break
         current = current.parent

      path.reverse()
      print("Path:", " -> ".join(path))
      print("Total cost:", total_cost)


In [3]:
class Queue:
    def __init__(self, mode='FIFO'):
        assert mode in ['FIFO', 'LIFO', 'PRIO'], "Mode must be 'FIFO', 'LIFO', or 'PRIO'"
        self.mode = mode
        self.items = []

    def push(self, item):
        if self.mode in ['FIFO', 'LIFO']:
            self.items.append(item)
        elif self.mode in ['PRIO']:
            self.items.append(item)
            self.items.sort(key=lambda x: x.value)
        
    def pop(self):
        if self.mode in ['FIFO', 'PRIO']:
            return self.items.pop(0)
        if self.mode in ['LIFO']:
            return self.items.pop()
                
    def is_empty(self):
        return len(self.items) == 0
    
    def contains(self, node):
      if isinstance(node, str):
         return any(n.name == node for n in self.items)
      if isinstance(node, Node):
         return any(n.name == node.name for n in self.items)

In [8]:
def BFS(graph:Graph, start:str, goal:str):
    
    print("BFS")
    
    #Check if the nodes are in the graph
    if start == goal or not graph.contains(start) or not graph.contains(goal):
        print("INVALID INPUT")
        return -1
    
    startNode = graph.get(start)
    goalNode = graph.get(goal)
        
    frontier = Queue('FIFO')
    frontier.push(startNode)
    explored = Queue()
    path: Queue = Queue()
    
    while not frontier.is_empty():
        node = frontier.pop()
        explored.push(node)
        
        parent = node
        
        for edge in node.edges:
  
            child = edge.end
            if not explored.contains(child) and not frontier.contains(child):
                if child == goalNode:
                    child.parent = parent
                    graph.print_path(child)
                    return graph
                
                child.parent = parent
                frontier.push(child)
    
    
    print("NODE NOT FOUND")            
    return -1

def DFS(graph:Graph, start:str, goal:str):
    
    print("DFS")
    
    if start == goal or not graph.contains(start) or not graph.contains(goal):
        print("INVALID INPUT")
        return -1
    
    startNode = graph.get(start)
    goalNode = graph.get(goal)
        
    frontier = Queue('LIFO')
    frontier.push(startNode)
    explored = Queue()
    path: Queue = Queue()
    
    while not frontier.is_empty():
        node = frontier.pop()
        explored.push(node)
        
        parent = node
        
        for edge in node.edges:
  
            child = edge.end
            if not explored.contains(child) and not frontier.contains(child):
                if child == goalNode:
                    child.parent = parent
                    graph.print_path(child)
                    return graph
                
                child.parent = parent
                frontier.push(child)
    
    
    print("NODE NOT FOUND")            
    return -1

def UCS(graph:Graph, start:str, goal:str):
    
    print("UCS")
    
    if start == goal or not graph.contains(start) or not graph.contains(goal):
        print("INVALID INPUT")
        return -1
    
    pathCost = 0
    startNode = graph.get(start)
    startNode.value = pathCost
    goalNode = graph.get(goal)
        
    frontier = Queue('PRIO')
    frontier.push(startNode)
    explored = Queue()
    path: Queue = Queue()
    
    
    while not frontier.is_empty():
        node = frontier.pop()
        explored.push(node)
        
        parent = node
        
        for edge in node.edges:
  
            child = edge.end
            child.value = node.value + edge.value
            if not explored.contains(child) and not frontier.contains(child):
                if child == goalNode:
                    child.parent = parent
                    graph.print_path(child)
                    return graph
                
                child.parent = parent
                frontier.push(child)
    
    
    print("NODE NOT FOUND")            
    return -1

In [9]:

romania = Graph( ['Or', 'Ne', 'Ze', 'Ia', 'Ar', 'Si', 'Fa',
 'Va', 'Ri', 'Ti', 'Lu', 'Pi', 'Ur', 'Hi',
 'Me', 'Bu', 'Dr', 'Ef', 'Cr', 'Gi'],
[
   ('Or', 'Ze', 71), ('Or', 'Si', 151),
   ('Ne', 'Ia', 87), ('Ze', 'Ar', 75),
   ('Ia', 'Va', 92), ('Ar', 'Si', 140),
   ('Ar', 'Ti', 118), ('Si', 'Fa', 99),
   ('Si', 'Ri', 80), ('Fa', 'Bu', 211),
   ('Va', 'Ur', 142), ('Ri', 'Pi', 97),
   ('Ri', 'Cr', 146), ('Ti', 'Lu', 111),
   ('Lu', 'Me', 70), ('Me', 'Dr', 75),
   ('Dr', 'Cr', 120), ('Cr', 'Pi', 138),
   ('Pi', 'Bu', 101), ('Bu', 'Gi', 90),
   ('Bu', 'Ur', 85), ('Ur', 'Hi', 98),
   ('Hi', 'Ef', 86)
] )
    
    #romania.print()
    
BFS(romania, 'Bu', 'Ti')
DFS(romania, 'Bu', 'Ti')
UCS(romania, 'Bu', 'Ti')

BFS
Path: Bu -> Fa -> Si -> Ar -> Ti
Total cost: 568
DFS
Path: Bu -> Pi -> Cr -> Dr -> Me -> Lu -> Ti
Total cost: 615
UCS
Path: Bu -> Pi -> Ri -> Si -> Ar -> Ti
Total cost: 536
